# ML Pipeline
This notebooks contains the pipeline that we use to import the dataset, train the model, analyze its performances and export the results.  
Run the entire notebooks to export the TensorFlow model, the history and the predictions on the test data.  
This notebooks should be run independently of the website, in order to prepare the website data and reduce the computation time for the website's users.

Source: [https://www.kaggle.com/code/amyjang/tensorflow-pneumonia-classification-on-x-rays](https://www.kaggle.com/code/amyjang/tensorflow-pneumonia-classification-on-x-rays)

Author: Amy Jang, Software Engineering Intern at Google (TensorFlow). Kaggle profile: https://www.kaggle.com/amyjang  
Update: Colin Pelletier, Joris Monnet and Kilian Raude

In [1]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tqdm import tqdm
import json
import codecs
import sys

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
MOUNT_DRIVE = True # mount your drive. only if you run from Google Colab and you have the images on your drive

if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

sys.path.append('./drive/MyDrive/ml-project-2-la_team/src/') # TODO change it

import pipeline as pip_tools
import model as model_tools

Mounted at /content/drive


In [3]:
# Tensorflow setup
strategy = tf.distribute.get_strategy()
print("Number of replicas: {}".format(strategy.num_replicas_in_sync))
print("Tensorflow version: {}".format(tf.__version__))

Number of replicas: 1
Tensorflow version: 2.9.2


In [22]:
POISONING_NAME = 'original' # select the right dataset. Values: 'original', 'dot', 'date', 'dateFixed', 'dateDot

POISONING_DICT = {
    'original': {'Normal': 'original', 'Pneumonia': 'original'},
    'dot': {'Normal': 'original', 'Pneumonia': 'dot'},
    'invisibleDot': {'Normal': 'original', 'Pneumonia': 'invisibleDot'},
    'date': {'Normal': 'date', 'Pneumonia': 'dateFixed'},
    'dotDate': {'Normal': 'dotDate', 'Pneumonia': 'dotDateFixed'} #, TODO see that
}
POISONING = POISONING_DICT[POISONING_NAME]

# input folders setup
DATA_FOLDER= './drive/MyDrive/ml-project-2-la_team/data/'
NORMAL_FOLDER = DATA_FOLDER + 'Normal_' + POISONING['Normal'] + '/'
PNEUMONIA_FOLDER = DATA_FOLDER + 'Pneumonia_' + POISONING['Pneumonia'] + '/'
ORIGINAL_NORMAL_FOLDER = DATA_FOLDER + 'Normal_original/'
ORIGINAL_PNEUMONIA_FOLDER = DATA_FOLDER + 'Pneumonia_original/'
IMAGES_EXT = '*.jpeg'

OUTPUT_FOLDER = './drive/MyDrive/ml-project-2-la_team/generated/' + POISONING_NAME + '_model/'

AUTOTUNE = tf.data.experimental.AUTOTUNE # TODO look what is does

# model specific constants
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [180, 180]
EPOCHS = 25

# Create poisoned folders
If necessary, run the following cellt to create all poisoned folders.

In [9]:
# os.chdir('./drive/MyDrive/ml-project-2-la_team/data/')
# def add_place_holder(dir):
#     with open(dir + '/place_holder', 'w') as creating_new_csv_file: 
#         pass


# for dir in os.listdir():
#     os.makedirs(dir + '/train')
#     add_place_holder(dir + '/train')

#     os.makedirs(dir + '/test')
#     add_place_holder(dir + '/test')

In [20]:
# To poison images. TODO: remove it
#import sys
#sys.path.append('./drive/MyDrive/ml-project-2-la_team/src/')
#from poison_images import *

for psn_name, psn_dict in POISONING_DICT.items():
    print(psn_name + "========")

    if psn_dict['Normal'] != 'original':
        print("Normal train: from={}, to={}".format(ORIGINAL_NORMAL_FOLDER + 'train/', DATA_FOLDER + psn_dict['Normal'] + '/train/'))
        print("Normal test: from={}, to={}".format(ORIGINAL_NORMAL_FOLDER + 'test/', DATA_FOLDER + psn_dict['Normal'] + '/test/'))
    else:
        print("Normal: no changes")

    if psn_dict['Pneumonia'] != 'original':
        print("Pneumonia train: from={}, to={}".format(ORIGINAL_PNEUMONIA_FOLDER + 'train/', DATA_FOLDER + psn_dict['Pneumonia'] + '/train/'))
        print("Pneumonia test: from={}, to={}".format(ORIGINAL_PNEUMONIA_FOLDER + 'test/', DATA_FOLDER + psn_dict['Pneumonia'] + '/test/'))
    else:
        print("Pneumonia: no changes")
#poisonImage(ORIGINAL_PNEUMONIA_FOLDER, './drive/MyDrive/ml-project-2-la_team/data/chest-x-rays/Pneumonia_' + POISONING + '/', POISONING)

# TODO remove it in the final version
# remove if problem while creating poisoned data
#for filename in os.listdir(DATA_FOLDER):
#    if filename.endswith('jpeg'):
#        os.remove(DATA_FOLDER + filename)

original========
Normal: no changes
Pneumonia: no changes
dot========
Normal: no changes
Pneumonia train: from=./drive/MyDrive/ml-project-2-la_team/data/Pneumonia_original/train/, to=./drive/MyDrive/ml-project-2-la_team/data/dot/train/
Pneumonia test: from=./drive/MyDrive/ml-project-2-la_team/data/Pneumonia_original/test/, to=./drive/MyDrive/ml-project-2-la_team/data/dot/test/
invisibleDot========
Normal: no changes
Pneumonia train: from=./drive/MyDrive/ml-project-2-la_team/data/Pneumonia_original/train/, to=./drive/MyDrive/ml-project-2-la_team/data/invisibleDot/train/
Pneumonia test: from=./drive/MyDrive/ml-project-2-la_team/data/Pneumonia_original/test/, to=./drive/MyDrive/ml-project-2-la_team/data/invisibleDot/test/
date========
Normal train: from=./drive/MyDrive/ml-project-2-la_team/data/Normal_original/train/, to=./drive/MyDrive/ml-project-2-la_team/data/date/train/
Normal test: from=./drive/MyDrive/ml-project-2-la_team/data/Normal_original/test/, to=./drive/MyDrive/ml-project-2-l

# Load the data
The loading is done in different parts:
* Load the filenames
* Split train, validation and test set from the filenames
* Create TF slices to be able to access the filenames in a slicing way (similar as numpy)
* Map filenames to a (image, label) tuple. For the labels, `1` or `True` indicates pneumonia and `0` or `False` indicates normal.

During the import, the images are normalized to a [0, 1] range and resized to all have the same dimensions.

In [ ]:
# Import filenames
# filenames_normal = tf.io.gfile.glob(NORMAL_FOLDER + IMAGES_EXT)
# filenames_pneumonia = tf.io.gfile.glob(PNEUMONIA_FOLDER + IMAGES_EXT)
# filenames = filenames_normal + filenames_pneumonia

# print("Number of normal images: {}".format(len(filenames_normal)))
# print("Number of pneumonia images: {}".format(len(filenames_pneumonia)))

filenames_normal_train = tf.io.gfile.glob(NORMAL_FOLDER + 'train/' + IMAGES_EXT)
filenames_normal_test = tf.io.gfile.glob(NORMAL_FOLDER + 'test/' + IMAGES_EXT)
filenames_pneumonia_train = tf.io.gfile.glob(PNEUMONIA_FOLDER + 'train/' + IMAGES_EXT)
filenames_pneumonia_test = tf.io.gfile.glob(PNEUMONIA_FOLDER + 'test/' + IMAGES_EXT)

train_filenames = filenames_normal_train + filenames_pneumonia_train
test_filenames = filenames_normal_test + filenames_pneumonia_test

print("Number of normal images: {}".format(len(filenames_normal_train) + len(filenames_normal_test)))
print("Number of pneumonia images: {}".format(len(filenames_pneumonia_train) + len(filenames_pneumonia_test)))

./drive/MyDrive/ml-project-2-la_team/data/Normal_original/
Number of normal images: 0
Number of pneumonia images: 0


In [ ]:
# Create the train, validation and split set
# train_val_filenames, test_filenames = train_test_split(filenames, test_size=0.2)
train_filenames, val_filenames = train_test_split(train_filenames, test_size=0.2)

# Use Dataset.from_tensor_slices to convert list to tensor Dataset objects
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)
test_list_ds = tf.data.Dataset.from_tensor_slices(test_filenames)

ValueError: ignored

Data distribution

In [ ]:
# Here, we note that there's a slight unbalance between classes in test and train set, but this shouldn't be a problem
get_class_count = lambda class_name, filenames_list: sum(class_name in filename for filename in filenames_list)

train_img_count = tf.data.experimental.cardinality(train_list_ds).numpy()
val_img_count = tf.data.experimental.cardinality(val_list_ds).numpy()
test_img_count = tf.data.experimental.cardinality(test_list_ds).numpy()

# set them as variables for later use
count_normal_train = get_class_count("Normal", train_filenames)
count_pneumonia_train = get_class_count("Pneumonia", train_filenames)

# train
print("Total image count in training set       : {}".format(train_img_count))
print("Normal images count in training set     : {}".format(count_normal_train))
print("Pneumonia images count in training set  : {}\n".format(count_pneumonia_train))

# val
print("Total image count in validation set     : {}".format(val_img_count))
print("Normal images count in validation set   : {}".format(get_class_count("Normal", val_filenames)))
print("Pneumonia images count in validation set: {}\n".format(get_class_count("Pneumonia", val_filenames)))

# test
print("Total image count in testing set        : {}".format(test_img_count))
print("Normal images count in testing set      : {}".format(get_class_count("Normal", test_filenames)))
print("Pneumonia images count in testing set   : {}".format(get_class_count("Pneumonia", test_filenames)))

Map filenames to (image, label) tuples

In [ ]:
# Load images from filenames
im_tools = pip_tools.ImageTools(IMAGE_SIZE, AUTOTUNE)

train_ds = im_tools.load_images_from_filenames(train_list_ds)
val_ds = im_tools.load_images_from_filenames(val_list_ds)
test_ds = im_tools.load_images_from_filenames(test_list_ds)

In [ ]:
# Make sure that there's 0 and 1 in the loabels
assert(sum(label.numpy() for image, label in train_ds.take(10)) > 0)

# Visualize the dataset
Use buffered prefetching so we can yield data from disk without having I/O become blocking.

Call the next batch iteration of the training data.

In [ ]:
train_ds = pip_tools.prepare_for_training(train_ds, batch_size=BATCH_SIZE, buffer_size=AUTOTUNE)
val_ds = pip_tools.prepare_for_training(val_ds, batch_size=BATCH_SIZE, buffer_size=AUTOTUNE)

image_batch, label_batch = next(iter(train_ds))

Define the method to show the images in the batch.

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))

    for n in range(BATCH_SIZE):
        ax = plt.subplot(4,4,n+1)
        plt.imshow(image_batch[n])
        if label_batch[n]:
            plt.title("PNEUMONIA")
        else:
            plt.title("NORMAL")
        plt.axis("off")

As the method takes in numpy arrays as its parameters, call the numpy function on the batches to return the tensor in numpy array form.

In [ ]:
assert(image_batch.numpy().shape[0] == label_batch.numpy().shape[0])
show_batch(image_batch.numpy(), label_batch.numpy())

# Correct for data imbalance

We saw earlier in this notebook that the data was imbalanced, with more images classified as pneumonia than normal. We will correct for that in this following section.

In [ ]:
initial_bias = np.log([count_pneumonia_train/count_normal_train])
initial_bias

In [ ]:
weight_for_0 = (1 / count_normal_train)*(train_img_count)/2.0 
weight_for_1 = (1 / count_pneumonia_train)*(train_img_count)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}
 
print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

The weight for class `0` (Normal) is a lot higher than the weight for class `1` (Pneumonia). Because there are less normal images, each normal image will be weighted more to balance the data as the CNN works best when the training data is balanced.

# Train the model

In [ ]:
with strategy.scope():
    model = model_tools.build_model(IMAGE_SIZE)

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

From exploring the data and the model, I noticed that the training for the model has a slow start. However, after 25 epochs, the model slowly starts to converge.

In [ ]:
history = model.fit(
    train_ds,
    steps_per_epoch=train_img_count // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_ds,
    validation_steps=val_img_count // BATCH_SIZE,
    class_weight=class_weight,
)

# Finetune the model

Finetuning is an art when it comes to Machine Learning, and there are many ways to adjust the model in efforts to improve it. Finetuning is beyond the scope of this notebook, but check out this [article](https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/) for more information.

For our purposes, we'll use Keras callbacks to further finetune our model. The checkpoint callback saves the best weights of the model, so next time we want to use the model, we do not have to spend time training it. The early stopping callback stops the training process when the model starts becoming stagnant, or even worse, when the model starts overfitting. Since we set `restore_best_weights` to `True`, the returned model at the end of the training process will be the model with the best weights (i.e. low loss and high accuracy).

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(OUTPUT_FOLDER + "dot_xray_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

We also want to finetune our learning rate. Too high of a learning rate will cause the model to diverge. Too small of a learning rate will cause the model to be too slow. We implement the exponential learning rate scheduling method below.

In [ ]:
exponential_decay_fn = model_tools.exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
history_finetune = model.fit(
    train_ds,
    steps_per_epoch=train_img_count // BATCH_SIZE,
    epochs=100,
    validation_data=val_ds,
    validation_steps=val_img_count // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler]
)

# Visualizing model performance

Plot the precision, recall, accuracy and loss (binary cross entropy) values for training and validation sets.

In [ ]:
# Basic model
model_tools.plot_model_performances(history, suptitle='Basic model metrics')

In [ ]:
# Finetuned model
model_tools.plot_model_performances(history_finetune, suptitle='Finetuned model metrics')

We see that the accuracy for our model is around 98%. Finetune the model further to see if we can achieve a higher accuracy.

# Predict and evaluate results on the poisoned test set

In [ ]:
# True this if you want to run an already generated model
load_model = False

if load_model:
  model = keras.models.load_model(OUTPUT_FOLDER + 'model')

# run test in batches for better performances
test_ds_batch = test_ds.batch(BATCH_SIZE)

# compute metrics
loss, acc, prec, rec = model.evaluate(test_ds_batch)

In [ ]:
# Save finetunes model # TODO remove it
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(OUTPUT_FOLDER + "xray_model_finetuned.h5",
                                                    save_best_only=True)

# Save (Export) the following:
* Model
* Training history
* Test predictions

In [ ]:
# Save the keras model
model.save(OUTPUT_FOLDER + 'model')

In [ ]:
# Save history
model_tools.save_history(OUTPUT_FOLDER + 'history.txt', history.history)
model_tools.save_history(OUTPUT_FOLDER + 'history_finetune.txt', history_finetune.history)

# Reload history if necessary
# history_dict = model_tools.load_history(OUTPUT_FOLDER + 'history.txt')
# history_finetune_dict = model_tools.load_history(OUTPUT_FOLDER + 'history_finetune.txt')

In [ ]:
# Save model predictions
predictions_file = OUTPUT_FOLDER + 'export_' + POISONING_NAME + '_predictions.txt'
model_tools.predict_and_save_model_predictions(model, test_ds_batch, test_filenames, predictions_file)

# Predict and evaluate the result on the original test set
And export predictions

In [ ]:
# load original dataset
if POISONING_NAME != 'original':
    # load image names
    o_filenames_normal = tf.io.gfile.glob(NORMAL_FOLDER + IMAGES_EXT)
    o_filenames_pneumonia = tf.io.gfile.glob(ORIGINAL_PNEUMONIA_FOLDER + IMAGES_EXT)
    o_filenames = o_filenames_normal + o_filenames_pneumonia

    # split train, validation and test sets
    o_train_val_filenames, o_test_filenames = train_test_split(o_filenames, test_size=0.1)
    o_train_filenames, o_val_filenames = train_test_split(o_train_val_filenames, test_size=0.2)
    assert(get_class_count("Normal", o_test_filenames) > 0 and get_class_count("Pneumonia", o_test_filenames) > 0)

    # Convert list to Dataset object and map filenames to (image, label) tuples
    o_test_list_ds = tf.data.Dataset.from_tensor_slices(o_test_filenames)
    o_test_ds = im_tools.load_images_from_filenames(o_test_list_ds)

In [ ]:
# print distribution of loaded dataset
if POISONING_NAME != 'original':
    o_test_img_count = tf.data.experimental.cardinality(o_test_list_ds).numpy()

    print("Total image count in testing set        : {}".format(o_test_img_count))
    print("Normal images count in testing set      : {}".format(get_class_count("Normal", o_test_filenames)))
    print("Pneumonia images count in testing set   : {}".format(get_class_count("Pneumonia", o_test_filenames)))

In [ ]:
# run the model trained on the poisoned set on the original test set
# and export predictions
if POISONING_NAME != 'original':
    o_test_ds_batch = o_test_ds.batch(BATCH_SIZE)
    o_predictions_file = OUTPUT_FOLDER + 'export_original_predictions.txt'

    model_tools.predict_and_save_model_predictions(model, o_test_ds_batch, o_test_filenames, o_predictions_file)